<a href="https://colab.research.google.com/github/Ario20/Melite/blob/main/Slope_Stats_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=G1jlUhH34WXwU1_RkgGFARKiqdTGYT0L6OgDgYzlUOM&tc=vpNYI3UwCAFcU4gnG8j53_vCk-7451q58LuLiHto36w&cc=psPj9lU3_CAOmUwJFvexmvrLZ95wl6J3IwGYLR3WTFE

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AdQt8qj7hAxSuH0rX6iZFaw4s8cnXIVK4h0DDFe13rR9ysnFdVRnDGdaF_o

Successfully saved authorization token.


In [2]:
import numpy as np
ee.Initialize()

In [3]:
def slope_correction(collection, elevation, model, buffer=0):
    '''This function applies the slope correction on a collection of Sentinel-1 data
       
       :param collection: ee.Collection of Sentinel-1
       :param elevation: ee.Image of DEM
       :param model: model to be applied (volume/surface)
       :param buffer: buffer in meters for layover/shadow amsk
        
        :returns: ee.Image
    '''
    
    def _volumetric_model_SCF(theta_iRad, alpha_rRad):
        '''Code for calculation of volumetric model SCF
        
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range
        
        :returns: ee.Image
        '''
        
        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi/180)
        
        # model
        nominator = (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).tan()
        denominator = (ninetyRad.subtract(theta_iRad)).tan()
        return nominator.divide(denominator) 
    
    
    def _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad):
        '''Code for calculation of direct model SCF
        
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range
        :param alpha_azRad: ee.Image of slope steepness in azimuth
        
        :returns: ee.Image
        '''
        
        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi/180)
        
        # model  
        nominator = (ninetyRad.subtract(theta_iRad)).cos()
        denominator = (alpha_azRad.cos()
          .multiply((ninetyRad.subtract(theta_iRad).add(alpha_rRad)).cos()))

        return nominator.divide(denominator)


    def _erode(image, distance):
      '''Buffer function for raster

      :param image: ee.Image that shoudl be buffered
      :param distance: distance of buffer in meters
        
      :returns: ee.Image
      '''
      
      d = (image.Not().unmask(1)
          .fastDistanceTransform(30).sqrt()
          .multiply(ee.Image.pixelArea().sqrt()))
    
      return image.updateMask(d.gt(distance))
    
    
    def _masking(alpha_rRad, theta_iRad, buffer):
        '''Masking of layover and shadow
        
        
        :param alpha_rRad: ee.Image of slope steepness in range
        :param theta_iRad: ee.Image of incidence angle in radians
        :param buffer: buffer in meters
        
        :returns: ee.Image
        '''
        # layover, where slope > radar viewing angle 
        layover = alpha_rRad.lt(theta_iRad).rename('layover')

        # shadow 
        ninetyRad = ee.Image.constant(90).multiply(np.pi/180)
        shadow = alpha_rRad.gt(ee.Image.constant(-1).multiply(ninetyRad.subtract(theta_iRad))).rename('shadow')
        
        # add buffer to layover and shadow
        if buffer > 0:
            layover = _erode(layover, buffer)   
            shadow = _erode(shadow, buffer)  

        # combine layover and shadow
        no_data_mask = layover.And(shadow).rename('no_data_mask')
        
        return layover.addBands(shadow).addBands(no_data_mask)
                        
        
    def _correct(image):
        '''This function applies the slope correction and adds layover and shadow masks
        
        '''
        
        # get the image geometry and projection
        geom = image.geometry()
        proj = image.select(1).projection()
        
        # calculate the look direction
        heading = (ee.Terrain.aspect(image.select('angle'))
                                     .reduceRegion(ee.Reducer.mean(), geom, 1000)
                                     .get('aspect'))
                   

        # Sigma0 to Power of input image
        sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

        # the numbering follows the article chapters
        # 2.1.1 Radar geometry 
        theta_iRad = image.select('angle').multiply(np.pi/180)
        phi_iRad = ee.Image.constant(heading).multiply(np.pi/180)
        
        # 2.1.2 Terrain geometry
        alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(np.pi/180).setDefaultProjection(proj).clip(geom)
        phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(np.pi/180).setDefaultProjection(proj).clip(geom)
        
        # we get the height, for export 
        height = elevation.setDefaultProjection(proj).clip(geom)
        
        # 2.1.3 Model geometry
        #reduce to 3 angle
        phi_rRad = phi_iRad.subtract(phi_sRad)

        # slope steepness in range (eq. 2)
        alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

        # slope steepness in azimuth (eq 3)
        alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

        # local incidence angle (eq. 4)
        theta_liaRad = (alpha_azRad.cos().multiply((theta_iRad.subtract(alpha_rRad)).cos())).acos()
        theta_liaDeg = theta_liaRad.multiply(180/np.pi)

        # 2.2 
        # Gamma_nought
        gamma0 = sigma0Pow.divide(theta_iRad.cos())
        gamma0dB = ee.Image.constant(10).multiply(gamma0.log10()).select(['VV', 'VH'], ['VV_gamma0', 'VH_gamma0'])
        ratio_gamma = (gamma0dB.select('VV_gamma0')
                        .subtract(gamma0dB.select('VH_gamma0'))
                        .rename('ratio_gamma0'))

        if model == 'volume':
            scf = _volumetric_model_SCF(theta_iRad, alpha_rRad)

        if model == 'surface':
            scf = _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)

        # apply model for Gamm0_f
        gamma0_flat = gamma0.divide(scf)
        gamma0_flatDB = (ee.Image.constant(10)
                         .multiply(gamma0_flat.log10())
                         .select(['VV', 'VH'],['VV_gamma0flat', 'VH_gamma0flat'])
                        )

        masks = _masking(alpha_rRad, theta_iRad, buffer)

        # calculate the ratio for RGB vis
        ratio_flat = (gamma0_flatDB.select('VV_gamma0flat')
                        .subtract(gamma0_flatDB.select('VH_gamma0flat'))
                        .rename('ratio_gamma0flat')
                     )

        return (image.rename(['VV_sigma0', 'VH_sigma0', 'incAngle'])
                      .addBands(gamma0dB)
                      .addBands(ratio_gamma)
                      .addBands(gamma0_flatDB)
                      .addBands(ratio_flat)
                      .addBands(alpha_rRad.rename('alpha_rRad'))
                      .addBands(alpha_azRad.rename('alpha_azRad'))
                      .addBands(phi_sRad.rename('aspect'))
                      .addBands(alpha_sRad.rename('slope'))
                      .addBands(theta_iRad.rename('theta_iRad'))
                      .addBands(theta_liaRad.rename('theta_liaRad'))
                      .addBands(masks)
                      .addBands(height.rename('elevation'))
                 )    
    
    # run and return correction
    return collection.map(_correct)

In [14]:
# geometry for AOI in Hollin Hill North Yorkshire
geometry = ee.Geometry.Polygon([[[-1.231403738843886, 54.291069000328001 ], [ -0.687709680649565, 54.289328250942106 ], [ -0.679296058617742, 53.914486883179535 ], [ -1.228212364969746, 53.913906633384236 ], [ -1.231403738843886, 54.291069000328001]]])

# Corine land cover (2018) of Hollin HIll
lc = ee.Image('users/kennethcassar/LC/CLC2018_Hollin').rename('landcover')

# filter Sentinel-1 collection for AOI and selected dates
s1Collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(geometry) \
        .filterDate('2019-01-01', '2019-12-31')


# path to dem 
dem = 'USGS/SRTMGL1_003'

# list of models
models = ['volume', 'surface']

# this is the scale we want the data to be sampled
scale = 10 

# loop through all combinations and export to drive
for model in models:
    
    # get the respective collection and bands and mosaic to a single image
    corrected_image = slope_correction(
        s1Collection, 
        ee.Image(dem), 
        model
    ).mosaic()

    # we get geometry and projection of the image
    proj = corrected_image.select(1).projection()
    geom = corrected_image.clip(geometry).select(1).geometry()

    # add lc and bring everything to same projection/geometry
    added_LC = corrected_image.addBands(lc)
    image_reprojected = added_LC.reproject(proj, scale=scale).clip(geom)
      
    # get the bandlist 
    bandlist = image_reprojected.bandNames().getInfo()
    
    # create an export job for each band
    for band in bandlist:
        task = ee.batch.Export.image.toDrive(
            image=image_reprojected.select(band).clip(geom),
            description=band,
            folder='slope_correction/{}_{}_buf_0'.format(dem.split('/')[-1], model),
            fileNamePrefix=band,
            region=geom.coordinates().getInfo(),
            scale=scale,
            maxPixels=1e12
            )
        task.start()
        print(task.status())

{'state': 'READY', 'description': 'VV_sigma0', 'creation_timestamp_ms': 1657189098493, 'update_timestamp_ms': 1657189098493, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'VLFFUGMQFP3YMKEU6ULHQM7J', 'name': 'projects/earthengine-legacy/operations/VLFFUGMQFP3YMKEU6ULHQM7J'}
{'state': 'READY', 'description': 'VH_sigma0', 'creation_timestamp_ms': 1657189099197, 'update_timestamp_ms': 1657189099197, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'GXXR2KT5NIT2SRHBBMY4VXW2', 'name': 'projects/earthengine-legacy/operations/GXXR2KT5NIT2SRHBBMY4VXW2'}
{'state': 'READY', 'description': 'incAngle', 'creation_timestamp_ms': 1657189099914, 'update_timestamp_ms': 1657189099914, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'MQ2WNW3KJNSSBXJDE3B5NGRR', 'name': 'projects/earthengine-legacy/operations/MQ2WNW3KJNSSBXJDE3B5NGRR'}
{'state': 'READY', 'description': 'VV_gamma0', 'creation_timestamp_ms': 1657189100680, 'update_timestamp_ms': 1657189100680, 'start_times

In [4]:
!apt-get -qq install -y libspatialindex-dev
!pip install -q --upgrade earthpy

Selecting previously unselected package libspatialindex4v5:amd64.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack .../libspatialindex4v5_1.8.5-5_amd64.deb ...
Unpacking libspatialindex4v5:amd64 (1.8.5-5) ...
Selecting previously unselected package libspatialindex-c4v5:amd64.
Preparing to unpack .../libspatialindex-c4v5_1.8.5-5_amd64.deb ...
Unpacking libspatialindex-c4v5:amd64 (1.8.5-5) ...
Selecting previously unselected package libspatialindex-dev:amd64.
Preparing to unpack .../libspatialindex-dev_1.8.5-5_amd64.deb ...
Unpacking libspatialindex-dev:amd64 (1.8.5-5) ...
Setting up libspatialindex4v5:amd64 (1.8.5-5) ...
Setting up libspatialindex-c4v5:amd64 (1.8.5-5) ...
Setting up libspatialindex-dev:amd64 (1.8.5-5) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 1.4 MB 8.3 MB/s 
 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import itertools

import numpy as np
import pandas as pd 
from scipy import stats
from scipy import optimize

import rasterio as rio
from rasterio.windows import Window

import matplotlib.pyplot as plt
import seaborn as sns
import earthpy.plot as ep
%matplotlib inline

In [7]:
list_of_layers = [
    'VV_gamma0', 'VH_gamma0',
    'VV_gamma0flat', 'VH_gamma0flat',
    'alpha_rRad', 'theta_liaRad', 'aspect', 
    'layover', 'shadow',     
    'landcover']

# paths to dem and model types
dem = 'SRTMGL1_003'
models = ['volume', 'surface']
buffers = [0]

modelDict = {}

# loop thorugh all combinations and put into dictionary
for model, buffer in itertools.product(models, buffers):

    key = '{}_{}_buf_{}'.format(dem, model, buffer)
    # here we read all layers into a dictionary
    dataDict = {}
    for layer in list_of_layers:
        with rio.open('/content/drive/My Drive/slope_correction {}/{}.tif'.format(key, layer)) as src:
            print('Reading ' + layer)
            dataDict[layer] = np.nan_to_num(src.read(window=Window(0, 340, 3000, 3000)))[0]
            print(dataDict[layer].shape)
    
    # write respective dataDict to our model dict, where different models are stored
    modelDict[key] = dataDict

Reading VV_gamma0
(3000, 3000)
Reading VH_gamma0
(3000, 3000)
Reading VV_gamma0flat
(3000, 3000)
Reading VH_gamma0flat
(3000, 3000)
Reading alpha_rRad
(3000, 3000)
Reading theta_liaRad
(3000, 3000)
Reading aspect
(3000, 3000)
Reading layover
(3000, 3000)
Reading shadow
(3000, 3000)
Reading landcover
(3000, 3000)
Reading VV_gamma0
(3000, 3000)
Reading VH_gamma0
(3000, 3000)
Reading VV_gamma0flat
(3000, 3000)
Reading VH_gamma0flat
(3000, 3000)
Reading alpha_rRad
(3000, 3000)
Reading theta_liaRad
(3000, 3000)
Reading aspect
(3000, 3000)
Reading layover
(3000, 3000)
Reading shadow
(3000, 3000)
Reading landcover
(3000, 3000)


In [11]:
def create_plot_aspect_against_backscatter(model, data, array, mask, stats_dict, outfile, gridsize):
    
    # getlayer info #
    polarisation = layer.split('_')[0]
    flat = layer.split('_')[1][-4:]
    if flat != 'flat':
        flat = False
    
    # fore and backslope lines
    look_angle = 33.089022518635176 # hardcoded 
    backslope = look_angle
    foreslope = backslope + 180
    vertical_y = np.linspace(-28, 8, 5)
    fs_x = 0 * vertical_y + foreslope
    bs_x = 0 * vertical_y + backslope
    
    # calculate mean line
    horizontal_x = np.linspace(0, 360, 10) 
    mean_y = 0 * horizontal_x + stats_dict['mean']
    
    if not flat:
        y_label = r'$\gamma^0$ [dB]'
    else:
        y_label = r'$\gamma^0_f$ [dB]'
    
    # check for 0s in aspect and update mask
    data['aspect'][data['aspect'] == 0] = np.nan
    mask = mask & np.isfinite(data['aspect'])
    aspect_deg_masked = np.subtract(to_deg(data['aspect'][mask]), 180)
    aspect_deg_masked = to_deg(data['aspect'][mask])

    # plot
    # surpress plotting, since we only want to save the files
    plt.ioff()
    X = sns.jointplot(aspect_deg_masked, array[mask], kind='hex', gridsize=(gridsize, gridsize))
    X.ax_joint.plot(horizontal_x, mean_y, 'k--', linewidth=.75)
    X.ax_joint.plot(fs_x, vertical_y, 'k--', linewidth=.75)
    X.ax_joint.plot(bs_x, vertical_y, 'k--', linewidth=.75)
    X.ax_joint.set_xlabel(r'Aspect angle $\phi_s$ [deg]', fontsize=14)
    X.ax_joint.set_ylabel(y_label,  fontsize=14)
    X.ax_joint.set_ylim(-30, 10)
    #X.ax_joint.set_xlim(-190, 185)
    X.ax_joint.set_xlim(-10, 365)
    
    # add textbox with ampl, mean and sd
    props = dict(boxstyle='round', facecolor='lightgrey', alpha=0.5)
    textstr = '\n'.join((
        r'$\mathrm{A}=%.2f$' % (stats_dict['amplitude'], ),
        r'$\mu=%.2f$' % (stats_dict['mean'], ),
        r'$\sigma=%.2f$' % (stats_dict['sd'], )))
    X.ax_joint.text(290, -28,textstr, fontsize=10, bbox=props)
    
    # add title
    if not flat:
        title = '{} Backscatter modulation by slopes'.format(polarisation)
    else:
        title = '{} Backscatter after Model {}'.format(polarisation, model)
        
    plt.suptitle(title, x=0.45, y=1.02, fontweight='bold', fontsize=14)
    
    # save
    plt.savefig(outfile, bbox_inches='tight', pad_inches=0.5)
    plt.close()

In [12]:
def create_plot_alpha_against_backscatter(model, data, array, mask, stats_dict, outfile, gridsize):
    
    # getlayer info #
    polarisation = layer.split('_')[0]
    flat = layer.split('_')[1][-4:]
    if flat != 'flat':
        flat = False
    
    # calculate mean line
    horizontal_x = np.linspace(-40, 40, 10) 
    mean_y = 0 * horizontal_x + stats_dict['mean']
    
    if not flat:
        y_label = r'$\gamma^0$ [dB]'
    else:
        y_label = r'$\gamma^0_f$ [dB]'
    
    # check for 0s in aspect and update mask
    data['alpha_rRad'][data['alpha_rRad'] == 0] = np.nan
    mask = mask & np.isfinite(data['alpha_rRad'])
    alpha_deg_masked = to_deg(data['alpha_rRad'][mask])

    # plot
    # surpress plotting, since we only want to save the files
    plt.ioff()
    X = sns.jointplot(alpha_deg_masked, array[mask], kind='hex', gridsize=(gridsize, gridsize))
    X.ax_joint.plot(horizontal_x, mean_y, 'k--', linewidth=.75)
    X.ax_joint.set_xlabel(r'Slope Steepness in range $\alpha$ [deg]', fontsize=14)
    X.ax_joint.set_ylabel(y_label, fontsize=14)
    X.ax_joint.set_ylim(-30, 10)
    X.ax_joint.set_xlim(-45, 45)
    
    # add textbox with ampl, mean and sd
    props = dict(boxstyle='round', facecolor='lightgrey', alpha=0.5)
    textstr = '\n'.join((
        r'$\mathrm{s}=%.2f$' % (stats_dict['slope'], ),
        r'$\mu=%.2f$' % (stats_dict['mean'], ),
        r'$\sigma=%.2f$' % (stats_dict['sd'], )))
    X.ax_joint.text(20, -28,textstr, fontsize=10, bbox=props)
    
    # add title
    if not flat:
        title = '{} Backscatter modulation by slopes'.format(polarisation)
    else:
        title = '{} Backscatter after Model {}'.format(polarisation, model)
        
    plt.suptitle(title, x=0.45, y=1.02, fontweight='bold', fontsize=14)
    
    # save
    plt.savefig(outfile, bbox_inches='tight', pad_inches=0.5)
    plt.close()

In [13]:
def create_plot_lia_against_backscatter(model, data, array, mask, stats_dict, outfile, gridsize):
    
    
    # getlayer info #
    polarisation = layer.split('_')[0]
    flat = layer.split('_')[1][-4:]
    if flat != 'flat':
        flat = False
    
    # calculate mean line
    horizontal_x = np.linspace(0, 90, 10) 
    mean_y = 0 * horizontal_x + stats_dict['mean']
    
    if not flat:
        y_label = r'$\gamma^0$ [dB]'
    else:
        y_label = r'$\gamma^0_f$ [dB]'
    
    # check for 0s in aspect and update mask
    data['theta_liaRad'][data['theta_liaRad'] == 0] = np.nan
    mask = mask & np.isfinite(data['theta_liaRad'])
    theta_deg_masked = to_deg(data['theta_liaRad'][mask])

    # plot
    # surpress plotting, since we only want to save the files
    plt.ioff()
    X = sns.jointplot(theta_deg_masked, array[mask], kind='hex', gridsize=(gridsize, gridsize))
    X.ax_joint.plot(horizontal_x, mean_y, 'k--', linewidth=.75)
    X.ax_joint.set_xlabel(r'Local Incidence Angle $\theta$ [deg]', fontsize=14)
    X.ax_joint.set_ylabel(y_label, fontsize=14)
    X.ax_joint.set_ylim(-30, 10)
    X.ax_joint.set_xlim(-10, 100)
    
    # add textbox with ampl, mean and sd
    props = dict(boxstyle='round', facecolor='lightgrey', alpha=0.5)
    textstr = '\n'.join((
        #r'$\mathrm{s}=%.2f$' % (stats_dict['slope'], ),
        r'$\mu=%.2f$' % (stats_dict['mean'], ),
        r'$\sigma=%.2f$' % (stats_dict['sd'], )))
    X.ax_joint.text(-5, -28,textstr, fontsize=10, bbox=props)
    
    # add title
    if not flat:
        title = '{} Backscatter modulation by slopes'.format(polarisation)
    else:
        title = '{} Backscatter after Model {}'.format(polarisation, model)
        
    plt.suptitle(title, x=0.45, y=1.02, fontweight='bold', fontsize=14)
    
    # save
    plt.savefig(outfile, bbox_inches='tight', pad_inches=0.5)
    plt.close()

In [14]:
# from rad to degree
def to_deg(rad):
    return np.multiply(rad,  np.divide(180,np.pi))

# sin function or curve fitting
def sin_func(x, a, b, c):
    return a * np.sin(x + b) - c

def tf_stats(data, array, layer, lc_class, mask):
    
    #------------------------------------------------
    # slope calculation
    
    # mask alpha angle
    data['alpha_rRad'][data['alpha_rRad'] == 0] = np.nan
    mask_alpha = mask & np.isfinite(data['alpha_rRad'])
    alpha_deg_masked = np.subtract(to_deg(data['alpha_rRad'][mask_alpha]), 180)
    
    # mask out nans
    x = array.copy()
    x[~mask_alpha] = np.nan
    x = x[np.logical_not(np.isnan(x))] 
    y = alpha_deg_masked
    y = y[np.logical_not(np.isnan(y))]
    
    # lin-regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(y, x)        
    #------------------------------------------------
    
    #------------------------------------------------
    # amplitude calculation
    
    # mask aspect 0s and nans
    data['aspect'][data['aspect'] == 0] = np.nan
    mask_aspect = mask & np.isfinite(data['aspect'])
    aspect_deg_masked = np.subtract(data['aspect'][mask_aspect], np.pi)
    
    # mask out nans
    x = array.copy()
    x[~mask_aspect] = np.nan
    x = x[np.logical_not(np.isnan(x))]
    y = aspect_deg_masked
    y = y[np.logical_not(np.isnan(y))]
   
    # curve fitting
    params, params_covariance = optimize.curve_fit(sin_func, y, x)
    amp = params[0]
    #------------------------------------------------

    # mean, sd
    mean = np.nanmean(array[mask_alpha])
    std = np.nanstd(array[mask_alpha])
    
    # create final dictionary
    stat_dict = {'lc_class': lc_class, 
                 'layer': layer, 
                 'count': np.sum(mask), 
                 'mean': mean, 
                 'sd': std, 
                 'slope': slope, 
                 'amplitude': np.abs(amp)
                }
    
    return stat_dict

In [15]:
# list of class names
legend_entries = ["NO DATA", "Discontinous urban fabric", "Industrial", "Road and rail network", "Airports", "Mineral extraction sites", "Green urban areas", 
				  "Sport facilities", "Non-irrigated arable land", "Pastures", "Broad-leaved forest", "Coniferous forest", "Mixed forest", "Natural Grassland", 
				  "Moors and Heathland", "Transitional woodland - shrub"]
# list of class values
ras_values = [0, 2, 3, 4, 6, 7, 10, 11, 12, 18, 23, 24, 25, 26, 27, 29]

# prepare columns for new dataframe
df_cols = ['lc_class', 'layer', 'count', 'mean', 'sd', 'slope', 'amplitude']

# paths to dem and model types and buffer
dem = 'SRTMGL1_003'
models = ['surface', 'volume']
buffers = [0] 

# loop through different models and buffers
for model, buffer in itertools.product(models, buffers):
    
    # get respective arrays within the model/datadict
    key = '{}_{}_buf_{}'.format(dem, model, buffer)
    dataDict = modelDict[key]
    
    print(' INFO: Creating figures and stats for {} {} with buffer {}.'
      .format(dem, model, buffer)
    )
    # create empty dataframe for statistics
    df_stats = pd.DataFrame(columns=df_cols)
    
    # crate outdirectory where plots and stats will be saved
    outdir = '/content/drive/My Drive/slope_correction/pictures/{}/'.format(key)
    os.makedirs(outdir, exist_ok=True)
    
    # loop through classes and respective raster values file
    for legend_entry, ras_value in zip(legend_entries, ras_values):
        
        # set raster value respective to class
        print(' INFO: Analysing {} with raster value {}.'
          .format(legend_entry, ras_value)
        )

        # loop thorugh different corrected and uncorrected layers
        for layer in ['VV_gamma0', 'VV_gamma0flat', 'VH_gamma0', 'VH_gamma0flat']:
                        
            # create combined Land Cover and Layover/Shadow mask
            valid_data_mask = (
                [dataDict['landcover'] == ras_value] & 
                (dataDict['layover'] > 0) & 
                (dataDict['shadow'] > 0)
            )[0] 

            # apply this mask and add valid data mask of backscatter array 
            array = dataDict[layer].copy()
            array[array == 0] = np.nan
            mask = valid_data_mask & np.isfinite(array)

            # set everything else to nan
            array[~mask] = np.nan

            # for some classes array might be empty, so we add an if
            if True in np.unique(np.isfinite(array)):
                
                # calculate stats
                stats_dict = tf_stats(
                    dataDict.copy(), array, layer, legend_entry, mask
                )
                stats_dict['lc_class_code'] =  ras_value

                # and put into pandas dataframe
                df = pd.DataFrame([stats_dict], columns=stats_dict.keys())
                df_stats = df_stats.append(stats_dict, ignore_index=True)

                # plotting
                gridsize=100
                model_nr = '1' if model == 'volume' else '2'

                create_plot_aspect_against_backscatter(
                    model_nr, dataDict.copy(), array, mask, stats_dict, 
                    '{}/{}_{}_vs_aspect.png'.format(outdir, legend_entry, layer), 
                    gridsize
                )

                create_plot_alpha_against_backscatter(
                    model_nr, dataDict.copy(), array, mask, stats_dict, 
                    '{}/{}_{}_vs_slope.png'.format(outdir, legend_entry, layer), 
                    gridsize
                )

                create_plot_lia_against_backscatter(
                    model_nr, dataDict.copy(), array, mask, stats_dict, 
                    '{}/{}_{}_vs_LIA.png'.format(outdir, legend_entry, layer), 
                    gridsize
                )

        # save the complete stas dataframe to pickle
        df_stats.reset_index()
        df_stats.to_pickle('{}/stats.pickle'.format(outdir)) 


 INFO: Creating figures and stats for SRTMGL1_003 surface with buffer 0.
 INFO: Analysing NO DATA with raster value 0.
 INFO: Analysing Discontinous urban fabric with raster value 2.


/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-package

 INFO: Analysing Industrial with raster value 3.


/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-package

 INFO: Analysing Road and rail network with raster value 4.
 INFO: Analysing Airports with raster value 6.
 INFO: Analysing Mineral extraction sites with raster value 7.
 INFO: Analysing Green urban areas with raster value 10.
 INFO: Analysing Sport facilities with raster value 11.


...

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-package

 INFO: Analysing Moors and Heathland with raster value 27.


/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-package

 INFO: Analysing Transitional woodland - shrub with raster value 29.


/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/usr/local/lib/python3.7/dist-package

In [18]:
df_stats_dict = {}

# paths to dem and model types
dem = 'SRTMGL1_003'
models = ['surface', 'volume'] 

# this is for the concatenation of VV and VH stats
concat_cols = ['lc_class', 'VV', 'Pixel count', 'VV mean', 'VV SD', 'VV slope', 'VV amp', 
                'VH', 'VH mean', 'VH SD', 'VH slope', 'VH amp']

# create empty list for adding all stats within the for loop
df_con_merged = []

# loop through different dems and models
for model in models:

    # get respective arrays within the datadict
    key = '{}_{}_buf_0'.format(dem, model)
    
    # read each df into the dictionary
    outdir = '/content/drive/My Drive/slope_correction/pictures/{}/'.format(key)
    df_stats = pd.read_pickle('{}/stats.pickle'.format(outdir))
    
    # split into vv and vh
    df_vv = df_stats[df_stats['layer'].str.contains('VV')].reset_index().rename(columns={'layer': 'VV-pol'})
    df_vh = df_stats[df_stats['layer'].str.contains('VH')].reset_index().rename(columns={'layer': 'VH-pol'})
    
   # this is for the concatenation of VV and VH stats
concat_cols = ['lc_class', 'lc class code', 'VV', 'Pixel count', 'VV mean', 'VV SD', 'VV slope', 'VV amp', 
                'VH', 'VH mean', 'VH SD', 'VH slope', 'VH amp']

# create empty list for adding all stats within the for loop
df_con_merged = []

# loop through different dems and models
for model in models:

    # get respective arrays within the datadict
    key = '{}_{}_buf_0'.format(dem, model)
    
    # read each df into the dictionary
    outdir = '/content/drive/My Drive/slope_correction/pictures/{}/'.format(key)
    df_stats = pd.read_pickle('{}/stats.pickle'.format(outdir))
    
    # split into vv and vh
    df_vv = df_stats[df_stats['layer'].str.contains('VV')].reset_index().rename(columns={'layer': 'VV-pol'})
    df_vh = df_stats[df_stats['layer'].str.contains('VH')].reset_index().rename(columns={'layer': 'VH-pol'})
    
    # concat vv and vh columns
    df_con = pd.concat([df_vv, df_vh], axis=1, ignore_index=True)
    
    # rename columns
    df_con.columns = ['i_2', 'lc_class', 'VV', 'Pixel count', 'VV mean', 'VV SD', 'VV slope', 'VV amp', 'lc class code',
                      'i_3', 'lc_class_2', 'VH', 'VH count', 'VH mean', 'VH SD', 'VH slope', 'VH amp', 'lc class code_2']
    
    # subset columns
    df_con = df_con[['lc_class', 'lc class code', 'VV', 'Pixel count', 'VV mean', 'VV SD', 'VV slope', 'VV amp', 
                     'VH', 'VH mean', 'VH SD', 'VH slope', 'VH amp']]
    
     # rename the layer names 
    df_con['VV'] = df_con['VV'].str.replace('VV_gamma0flat', '{}'.format(key))
    df_con['VH'] = df_con['VH'].str.replace('VH_gamma0flat', '{}'.format(key))
    
    df_con['VV'] = df_con['VV'].str.replace('VV_gamma0', 'Original')
    df_con['VH'] = df_con['VH'].str.replace('VH_gamma0', 'Original')
    
    # merge to existent dfs
    df_con_merged.append(df_con)


# bring all the data together
appended_data = pd.concat(df_con_merged)

# exclude marginal classes
appended_data = appended_data[appended_data['Pixel count'] >= 100000]
appended_data = appended_data[['lc_class', 'VV', 'VV mean', 'VV SD', 'VV slope', 'VV amp', 'VH mean', 'VH SD', 'VH slope', 'VH amp']]
    

# rename model names
appended_data['VV'] = appended_data['VV'].str.replace('SRTMGL1_003_volume_buf_0', 'Volume model')
appended_data['VV'] = appended_data['VV'].str.replace('SRTMGL1_003_surface_buf_0', 'Direct model')

# remove double entries for original data and reindex 
appended_data.drop_duplicates(subset=['lc_class', 'VV'], keep='first', inplace=True)
appended_data = appended_data.set_index(['lc_class', 'VV']).sort_values(['lc_class', 'VV'], ascending=False)

# rename model column
appended_data

VV mean     VV SD  VV slope  \
lc_class                  VV                                            
Sport facilities          Volume model  -9.943875  3.339057 -0.006815   
                          Original      -9.943762  3.401402  0.138396   
                          Direct model  -9.983468  3.338685  0.047198   
Pastures                  Volume model  -9.541957  2.896123  0.042378   
                          Original      -9.556830  2.942736  0.169450   
                          Direct model  -9.568624  2.906711  0.087481   
Non-irrigated arable land Volume model  -8.119613  2.875620 -0.013746   
                          Original      -8.138707  2.886054  0.113625   
                          Direct model  -8.137921  2.876754  0.033869   
Moors and Heathland       Volume model -10.105102  2.525421 -0.030970   
                          Original     -10.111744  2.526741  0.102389   
                          Direct model -10.115244  2.520343  0.019602   
Discontinous urban fabric Volume model  -7.292254  3.446096 -0.031223   
                          Original      -7.283939  3.451254  0.102848   
                          Direct model  -7.297059  3.446790  0.018898   
Coniferous forest         Volume model  -8.130899  2.222362 -0.005558   
                          Original      -8.155971  2.340565  0.130922   
                          Direct model  -8.207026  2.231851  0.040917   
Broad-leaved forest       Volume model  -8.025382  2.588062  0.018598   
                          Original      -7.734499  3.079845  0.187601   
                          Direct model  -8.093039  2.663665  0.073391   

                                          VV amp    VH mean     VH SD  \
lc_class                  VV                                            
Sport facilities          Volume model  0.205701 -16.421189  3.782416   
                          Original      0.453049 -16.421077  3.851071   
                          Direct model  0.083224 -16.460782  3.781378   
Pastures                  Volume model  0.167646 -16.523067  3.045254   
                          Original      0.573229 -16.537940  3.091237   
                          Direct model  0.304343 -16.549735  3.052435   
Non-irrigated arable land Volume model  0.230516 -15.849866  2.869607   
                          Original      0.285586 -15.868960  2.882720   
                          Direct model  0.191280 -15.868174  2.869716   
Moors and Heathland       Volume model  0.163050 -15.837625  2.840344   
                          Original      0.250345 -15.844267  2.842624   
                          Direct model  0.098864 -15.847767  2.836005   
Discontinous urban fabric Volume model  0.316968 -14.858618  3.001964   
                          Original      0.319275 -14.850304  3.015299   
                          Direct model  0.278198 -14.863424  3.004949   
Coniferous forest         Volume model  0.279604 -13.427549  2.410555   
                          Original      0.742029 -13.452621  2.520262   
                          Direct model  0.296064 -13.503676  2.419836   
Broad-leaved forest       Volume model  0.661096 -13.359874  2.871961   
                          Original      2.070065 -13.068991  3.329825   
                          Direct model  1.043936 -13.427531  2.936351   

                                        VH slope    VH amp  
lc_class                  VV                                
Sport facilities          Volume model  0.008058  0.159671  
                          Original      0.153268  0.687092  
                          Direct model  0.062071  0.305054  
Pastures                  Volume model  0.048142  0.174899  
                          Original      0.175214  0.589627  
                          Direct model  0.093245  0.317989  
Non-irrigated arable land Volume model -0.002861  0.112336  
                          Original      0.124510  0.282903  
                          Direct model  0.044754  0.098848  
Moors and H